In [2]:
import torch
import temp
import torch.nn as nn
import torchvision.models as model
from torchvision import transforms as trn
import os
import sys
from PIL import Image
from torch.autograd import Variable as V
import torchvision.models as models
from torch.nn import functional as F
import torch.optim as optim
from sklearn.svm import SVC
from torch.utils.data import DataLoader
from copy import deepcopy

from torch.autograd import Variable
from torchvision import datasets, models, transforms

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import shuffle

import numpy as np

mean = [0.4856586910840433, 0.4856586910840433, 0.4856586910840433]
std = [0.14210993338737993, 0.14210993338737993, 0.14210993338737993]

In [3]:
learning_milestones = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95,]
model = models.__dict__['resnet50'](num_classes=365)
checkpoint = torch.load('./resnet50_places365.pth.tar', map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomSizedCrop(220),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'validation': transforms.Compose([
        transforms.Scale(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py:563: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")
/usr/local/lib/python3.5/dist-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [6]:
#load data
dsets = {
    x : datasets.ImageFolder(os.path.join('./place365/indoorCVPR_09_MIT67/', x), data_transforms[x])
    for x in ['train','validation']
}

dset_loaders = {
    x : torch.utils.data.DataLoader(dsets[x], batch_size =190, shuffle=True)
    for x in ['train','validation']
}

dset_sizes = {x: len(dset_loaders[x]) for x in ['train','validation']}
dataset_sizes = {x: len(dsets[x]) for x in ['train', 'validation']}
dset_classes = dsets['train'].classes
dset_classes_val = dsets['validation'].classes

60
33
11310
6240
['airport_inside', 'artstudio', 'auditorium', 'bakery', 'bar', 'bathroom', 'bedroom', 'bookstore', 'bowling', 'buffet', 'casino', 'children_room', 'church_inside', 'classroom', 'cloister', 'closet', 'clothingstore', 'computerroom', 'concert_hall', 'corridor', 'deli', 'dentaloffice', 'dining_room', 'elevator', 'fastfood_restaurant', 'florist', 'gameroom', 'garage', 'greenhouse', 'grocerystore', 'gym', 'hairsalon', 'hospitalroom', 'inside_bus', 'inside_subway', 'jewelleryshop', 'kindergarden', 'kitchen', 'laboratorywet', 'laundromat', 'library', 'livingroom', 'lobby', 'locker_room', 'mall', 'meeting_room', 'movietheater', 'museum', 'nursery', 'office', 'operating_room', 'pantry', 'poolinside', 'prisoncell', 'restaurant', 'restaurant_kitchen', 'shoeshop', 'stairscase', 'studiomusic', 'subway', 'toystore', 'trainstation', 'tv_studio', 'videostore', 'waitingroom', 'warehouse', 'winecellar']
['airport_inside', 'artstudio', 'auditorium', 'bakery', 'bar', 'bathroom', 'bedroom'

In [6]:
criterion = nn.CrossEntropyLoss()
model = model.cuda()
model.eval()

model = torch.nn.DataParallel(model)

In [7]:
class BYNET(nn.Module):
    def __init__(self, num_classes=15):
        super(BYNET, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=(14,14), stride=(14,14))
        )
        self.layer_relu = nn.Sequential(
            nn.ReLU(),
            nn.Linear(512*4, 1024)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=(7, 7), stride=(7,7))
        )
        self.layer2_relu = nn.Sequential(
            nn.ReLU(),
            nn.Linear(1024*4, 1024)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(2048, 2048, kernel_size=(4, 4), stride=(3,3))          
        )
        self.layer3_relu = nn.Sequential(
            nn.ReLU(),
            nn.Linear(2048*4, 1024)
        )
        
        self.fc1 = nn.Linear(1024*3,1024)
        self.fc2 = nn.Linear(1024, 395)
    
    def forward(self, x):
        out1 = self.layer1(x[0])
        out1 = out1.view(-1, num_flat_features(out1))
        out1 = self.layer_relu(out1) 
        
        out2 = self.layer2(x[1])
        out2 = out2.view(-1, num_flat_features(out2))
        out2 = self.layer2_relu(out2) 
        
        out3 = self.layer3(x[2])
        out3 = out3.view(-1, num_flat_features(out3))
        out3 = self.layer3_relu(out3) 
        
        out = torch.cat((out1,out2,out3),1)
        out = self.fc1(out)
        out = self.fc2(out)
        
        return out

In [2]:
def num_flat_features(x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [8]:
def train_model(model_ft,optimizer,scheduler,net,num_epochs):
    
    best_model_wts = deepcopy(net.state_dict())
    best_acc = 0.0
        
    for epoch in range(1,num_epochs+1):
        for phase in ['train','validation']:
            if phase == 'train':
                scheduler.step()
                print('\n=> Training Epoch #%d' %(epoch))
                net.train()
            else:
                print('\n=> Validation Epoch #%d' %(epoch))
                net.eval()

            running_loss = 0.0       
            running_corrects = 0
            tot = 0.0

            for batch_idx, (inputs, target) in enumerate(dset_loaders[phase]):
                inputs, target = Variable(inputs.cuda()), Variable(target.cuda())

                #추가된 layer 이외에 다른 layer는 학습 x
                for param in model.parameters():
                    param.requires_grad = False

                optimizer.zero_grad()


                out1 = model_ft.module.conv1.forward(inputs)
                out2 = model_ft.module.relu(model_ft.module.bn1(out1))
                out3 = model_ft.module.maxpool.forward(out2)

                layer1_feature = model_ft.module.layer1.forward(out3)
                layer2_feature = model_ft.module.layer2.forward(layer1_feature)
                layer3_feature = model_ft.module.layer3.forward(layer2_feature)
                layer4_feature = model_ft.module.layer4.forward(layer3_feature)

                layer_feature_list = list()
                layer_feature_list.append(layer2_feature)
                layer_feature_list.append(layer3_feature)
                layer_feature_list.append(layer4_feature)

                output = net.forward(layer_feature_list)    
                _,preds = torch.max(output.data,1)
                loss  = criterion(output, target)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item()
                running_corrects += preds.eq(target.data).cpu().sum()
                tot += target.size(0)

                if phase == 'train':
                    sys.stdout.write('\r')
                    sys.stdout.write('| Epoch [%2d/%2d] Iter [%3d/%3d]\t\tLoss %.4f\tAcc %.2f%%'
                            %(epoch, num_epochs, batch_idx,
                                (len(dsets[phase])//190)+1, loss.data[0], 100.*running_corrects/tot))
                    sys.stdout.flush()
                    sys.stdout.write('\r')
                if phase == 'validation':
                    print("===> Validation Epoch[{}]({}/{}): Loss: {:.4f}: Accuracy: {:.4f}".format(epoch, batch_idx, dset_sizes['validation'], loss.data[0],100.*running_corrects/tot))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'validation' and epoch_acc > best_acc:
                print('\n| Validation Epoch #%d\t\t\tLoss %.4f\tAcc %.2f%%'
                    %(epoch, loss.data[0], 100.*epoch_acc))
                best_acc = epoch_acc
                best_model_wts = deepcopy(net.state_dict())

                if not os.path.isdir('checkpoint'):
                    os.mkdir('checkpoint')
                save_point = './checkpoint/'
                if not os.path.isdir(save_point):
                    os.mkdir(save_point)
                torch.save(best_model_wts, '{}.pth'.format('BYNET_best_sun397'))
        
        net_copy = deepcopy(net.state_dict())
        torch.save(net, '{}.pth'.format(epoch))
        
    print('Best validation Acc\t{:.2f}%'.format(best_acc*100))
    
    return best_model_wts

In [ ]:
net = BYNET().cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1,weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=learning_milestones,gamma=0.8)
net_ft = train_model(model, optimizer,scheduler,net,100)


=> Training Epoch #1


/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 150 bytes but only got 128. Skipping tag 37510
  " Skipping tag %s" % (size, len(data), tag))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:61: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 11796480 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 811428571 bytes but only got 0. Skipping tag 34464
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1572864 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting 

...

===> Validation Epoch[64](21/162): Loss: 1.3026: Accuracy: 61.0000
===> Validation Epoch[64](22/162): Loss: 1.3085: Accuracy: 61.0000
===> Validation Epoch[64](23/162): Loss: 1.4902: Accuracy: 61.0000
===> Validation Epoch[64](24/162): Loss: 1.5899: Accuracy: 61.0000
===> Validation Epoch[64](25/162): Loss: 1.4613: Accuracy: 61.0000
===> Validation Epoch[64](26/162): Loss: 1.5095: Accuracy: 61.0000
===> Validation Epoch[64](27/162): Loss: 1.5016: Accuracy: 60.0000
===> Validation Epoch[64](28/162): Loss: 1.3466: Accuracy: 61.0000
===> Validation Epoch[64](29/162): Loss: 1.3111: Accuracy: 61.0000
===> Validation Epoch[64](30/162): Loss: 1.5108: Accuracy: 61.0000
===> Validation Epoch[64](31/162): Loss: 1.4906: Accuracy: 60.0000
===> Validation Epoch[64](32/162): Loss: 1.4352: Accuracy: 60.0000
===> Validation Epoch[64](33/162): Loss: 1.4156: Accuracy: 60.0000
===> Validation Epoch[64](34/162): Loss: 1.3831: Accuracy: 60.0000
===> Validation Epoch[64](35/162): Loss: 1.5485: Accuracy: 60.

===> Validation Epoch[64](143/162): Loss: 1.2125: Accuracy: 61.0000
===> Validation Epoch[64](144/162): Loss: 1.4873: Accuracy: 60.0000
===> Validation Epoch[64](145/162): Loss: 1.4127: Accuracy: 60.0000
===> Validation Epoch[64](146/162): Loss: 1.5467: Accuracy: 60.0000
===> Validation Epoch[64](147/162): Loss: 1.3539: Accuracy: 60.0000
===> Validation Epoch[64](148/162): Loss: 1.3988: Accuracy: 60.0000
===> Validation Epoch[64](149/162): Loss: 1.4405: Accuracy: 60.0000
===> Validation Epoch[64](150/162): Loss: 1.4728: Accuracy: 60.0000
===> Validation Epoch[64](151/162): Loss: 1.3923: Accuracy: 60.0000
===> Validation Epoch[64](152/162): Loss: 1.4065: Accuracy: 60.0000
===> Validation Epoch[64](153/162): Loss: 1.3483: Accuracy: 60.0000
===> Validation Epoch[64](154/162): Loss: 1.3279: Accuracy: 60.0000
===> Validation Epoch[64](155/162): Loss: 1.4620: Accuracy: 60.0000
===> Validation Epoch[64](156/162): Loss: 1.3832: Accuracy: 60.0000
===> Validation Epoch[64](157/162): Loss: 1.3788

===> Validation Epoch[65](100/162): Loss: 1.4735: Accuracy: 61.0000
===> Validation Epoch[65](101/162): Loss: 1.3861: Accuracy: 61.0000
===> Validation Epoch[65](102/162): Loss: 1.3338: Accuracy: 61.0000
===> Validation Epoch[65](103/162): Loss: 1.3476: Accuracy: 61.0000
===> Validation Epoch[65](104/162): Loss: 1.5474: Accuracy: 61.0000
===> Validation Epoch[65](105/162): Loss: 1.3108: Accuracy: 61.0000
===> Validation Epoch[65](106/162): Loss: 1.5032: Accuracy: 61.0000
===> Validation Epoch[65](107/162): Loss: 1.3219: Accuracy: 61.0000
===> Validation Epoch[65](108/162): Loss: 1.3591: Accuracy: 61.0000
===> Validation Epoch[65](109/162): Loss: 1.1793: Accuracy: 61.0000
===> Validation Epoch[65](110/162): Loss: 1.5834: Accuracy: 61.0000
===> Validation Epoch[65](111/162): Loss: 1.3482: Accuracy: 61.0000
===> Validation Epoch[65](112/162): Loss: 1.3186: Accuracy: 61.0000
===> Validation Epoch[65](113/162): Loss: 1.5510: Accuracy: 61.0000
===> Validation Epoch[65](114/162): Loss: 1.2993

===> Validation Epoch[66](56/162): Loss: 1.4598: Accuracy: 61.0000
===> Validation Epoch[66](57/162): Loss: 1.7227: Accuracy: 61.0000
===> Validation Epoch[66](58/162): Loss: 1.6042: Accuracy: 61.0000
===> Validation Epoch[66](59/162): Loss: 1.4365: Accuracy: 61.0000
===> Validation Epoch[66](60/162): Loss: 1.1444: Accuracy: 61.0000
===> Validation Epoch[66](61/162): Loss: 1.3439: Accuracy: 61.0000
===> Validation Epoch[66](62/162): Loss: 1.3091: Accuracy: 61.0000
===> Validation Epoch[66](63/162): Loss: 1.2894: Accuracy: 61.0000
===> Validation Epoch[66](64/162): Loss: 1.6086: Accuracy: 61.0000
===> Validation Epoch[66](65/162): Loss: 1.4868: Accuracy: 61.0000
===> Validation Epoch[66](66/162): Loss: 1.1767: Accuracy: 61.0000
===> Validation Epoch[66](67/162): Loss: 1.3942: Accuracy: 61.0000
===> Validation Epoch[66](68/162): Loss: 1.3959: Accuracy: 61.0000
===> Validation Epoch[66](69/162): Loss: 1.3918: Accuracy: 61.0000
===> Validation Epoch[66](70/162): Loss: 1.5987: Accuracy: 61.

===> Validation Epoch[67](13/162): Loss: 1.5039: Accuracy: 63.0000
===> Validation Epoch[67](14/162): Loss: 1.2121: Accuracy: 63.0000
===> Validation Epoch[67](15/162): Loss: 1.2874: Accuracy: 63.0000
===> Validation Epoch[67](16/162): Loss: 1.3872: Accuracy: 63.0000
===> Validation Epoch[67](17/162): Loss: 1.4952: Accuracy: 63.0000
===> Validation Epoch[67](18/162): Loss: 1.5115: Accuracy: 63.0000
===> Validation Epoch[67](19/162): Loss: 1.5308: Accuracy: 62.0000
===> Validation Epoch[67](20/162): Loss: 1.5084: Accuracy: 62.0000
===> Validation Epoch[67](21/162): Loss: 1.6984: Accuracy: 61.0000
===> Validation Epoch[67](22/162): Loss: 1.4307: Accuracy: 62.0000
===> Validation Epoch[67](23/162): Loss: 1.3101: Accuracy: 61.0000
===> Validation Epoch[67](24/162): Loss: 1.3431: Accuracy: 62.0000
===> Validation Epoch[67](25/162): Loss: 1.4438: Accuracy: 62.0000
===> Validation Epoch[67](26/162): Loss: 1.6519: Accuracy: 62.0000
===> Validation Epoch[67](27/162): Loss: 1.5124: Accuracy: 62.